# Preprocesamiento del dataset

Se seleccionó el dataset "Shoe vs Sandal vs Boot" extraído de Kaggle:
https://www.kaggle.com/datasets/hasibalmuzdadid/shoe-vs-sandal-vs-boot-dataset-15k-images

El dataset consiste en 3 carpetas con 5000 imágenes de cada clase. Se utilizará una CNN para clasificarlas en las respectivas 3 clases de calzado. 

Sin embargo, se requirió realizar un preprocesamiento para separar las imágenes en sus respectivos sets de entrenamiento, test y pruebas. 

Dicho código se proporciona en este archivo y se busca que puede ser útil en caso de que se requiera nuevamente realizar una partición del dataset.


> Imports

In [ ]:
import os
import shutil
import os.path
import numpy as np
from os import path
from google.colab import drive
from sklearn.model_selection import train_test_split

%matplotlib inline

 > Descargar el JSON que contiene el key para conectarse a mi usuario de Kaggle:

In [ ]:
!wget https://transfer.sh/QvI5Cp/kaggle.json

--2022-11-02 17:53:23--  https://transfer.sh/QvI5Cp/kaggle.json
Resolving transfer.sh (transfer.sh)... 144.76.136.153, 2a01:4f8:200:1097::2
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72 [application/json]
Saving to: ‘kaggle.json’

kaggle.json         100%[===================>]      72  --.-KB/s    in 0s      

2022-11-02 17:53:24 (9.50 MB/s) - ‘kaggle.json’ saved [72/72]



In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Verificar que la conexión a Kaggle sea exitosa
!kaggle datasets list

ref                                                             title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ------------------------------------------  ------  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                               9KB  2022-10-06 08:55:25           8467        274  1.0              
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset        11KB  2022-09-30 04:05:11           6736        220  1.0              
whenamancodes/students-performance-in-exams                     Students Performance in Exams                  9KB  2022-09-14 15:14:54          11867        218  1.0              
dimitryzub/walmart-coffee-listings-from-500-stores              Walmart Coffee Listings from 50

> Descargar el dataset utilizando el comando de la API de Kaggle

In [ ]:
!kaggle datasets download -d hasibalmuzdadid/shoe-vs-sandal-vs-boot-dataset-15k-images

 96% 45.0M/47.0M [00:02<00:00, 25.6MB/s]
100% 47.0M/47.0M [00:02<00:00, 17.2MB/s]


In [ ]:
# Descomprimir el archivo
!unzip -qq shoe-vs-sandal-vs-boot-dataset-15k-images.zip

> Extraer los nombres de los archivos de las imágenes, así como la clase a la que pertenecen para separarlos en set de entrenamiento, pruebas y validación

In [ ]:
# Los nombres y sus labels se guardarán de forma ordenada en dos listas que se transformarán posteriormente en arreglos.
image_directories = []
labels = []
# En la siguiente variable se debe cambiar el directorio donde se tenga el dataset
current_directory = '/content/Shoe vs Sandal vs Boot Dataset'

for class_name in os.listdir(current_directory):
  # Iterar sobre los folders de las clases: Boot-Shoe-Sandal y generar el path actualizado para el siguiente ciclo for
  class_directory = os.path.join(current_directory, class_name)
  for file in os.listdir(class_directory):
    # Iterar sobre los archivos dentro del path del folder de cada clase
    if file.endswith('.jpg'):
      # Concatenar el path y añadirlo a la lista, así como su respectivo label
      image_directories.append(os.path.join(class_directory, file))
      labels.append(class_name)

# Transformar las listas en arreglos para utilizar el método train_test_split
image_directories = np.array(image_directories)
labels = np.array(labels)

> Separar las imágenes en sets de entrenamiento, validación y pruebas

In [ ]:
# Definir la proporción de imágenes del dataset de pruebas (20% del dataset total en este caso)
test_ratio = 0.2
# Generar los conjuntos de test y train, stratify permite buscar que las clases estén balanceadas
img_train, img_test, labels_train, labels_test = train_test_split(image_directories, labels, test_size=test_ratio, shuffle = True, stratify = labels)

In [ ]:
# Definir la proporción de imágenes del dataset de validación (20% del dataset de entrenamiento en este caso)
validation_ratio = 0.2
# Generar los conjuntos de validation y train, stratify permite buscar que las clases estén balanceadas
img_train, img_val, labels_train, labels_val = train_test_split(img_train, labels_train, test_size = validation_ratio, shuffle = True, stratify = labels_train)

In [ ]:
# Cantidad de imágenes por conjunto de datos
print('TRAIN DATASET SIZE:', img_train.size)
print('VALIDATION DATASET SIZE:', img_val.size)
print('TEST DATASET SIZE:', img_test.size)

TRAIN DATASET SIZE: 9600
VALIDATION DATASET SIZE: 2400
TEST DATASET SIZE: 3000


> Crear directorio en Google Drive para utilizar image_dataset_from_directory

En lugar de tener un arreglo con todas los arreglos de imágenes, se han generado 6 arreglos (3 con los paths de las imágenes y 3 con sus respectivos labels) con la información necesaria para organizar los archivos a un nuevo directorio.

Este nuevo directorio se localiza en Google Drive con el objetivo de reutilizarlo si el entorno se desconecta. Sin embargo, es posible crearlo en este entorno de ejecución al modificar las variables que definen el directorio destino.

Esto permite utilizar el uso de memoria RAM.

In [ ]:
# Conectar cuenta de Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Directorio destino
current_drive_directory = '/content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset'

# Navegar hacia el directorio destino
os.chdir(current_drive_directory)
# Print Current Directory
!pwd
# Crear los directorios por cada conjunto de datos de entrenamiento, validación y pruebas
!mkdir train # Entrenamiento
!mkdir val # Validación
!mkdir test # Pruebas

# Listar Directorios
!ls

# Esta lista con las clases se utiliza para generar subdirectorios dentro de los directorios de test, train y val
class_directory = ['Boot', 'Shoe', 'Sandal']

# Crear subdirectorios
for folder in os.listdir(current_drive_directory):
  for class_name in class_directory:
    os.mkdir(os.path.join(current_drive_directory, folder, class_name))

/content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset
test  train  val


> Utilizar shutil para copiar las imágenes a sus directorios destino

In [ ]:
# Iterar sobre cada set para copiar los archivos 
image_sets = [img_train, img_val, img_test]
labels_sets = [labels_train, labels_val, labels_test]
# Declarar los nombres de los folders destino
folders = ['train', 'val', 'test']
for x in range(0,3):
  # Permite iterar sobre los labels
  label_iterator = 0
  for image in image_sets[x]:
    # Copiar la imagen a su respectivo nuevo path construido usando el label para indicar la carpeta en la que debe estar
    shutil.copy(image, os.path.join(current_drive_directory, folders[x], labels_sets[x][label_iterator]))
    label_iterator+=1

> Generar un zip con el dataset final (dataset-a0150480.zip)

In [ ]:
os.chdir('/content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning')

In [ ]:
!zip -r '/content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/dataset-a01750480.zip' '/content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset'

Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset/val/Shoe/Shoe (1550).jpg (deflated 16%)
  adding: content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset/val/Shoe/Shoe (3624).jpg (deflated 11%)
  adding: content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset/val/Shoe/Shoe (954).jpg (deflated 4%)
  adding: content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset/val/Shoe/Shoe (3919).jpg (deflated 8%)
  adding: content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset/val/Shoe/Shoe (1759).jpg (deflated 5%)
  adding: content/drive/Shareddrives/Concentracion/Bloque II/Momento de Retroalimentacion/Deep Learning/Dataset/val/Shoe/Shoe (2402).jpg (deflated 5%)
  adding: content/drive/Shareddr

Este dataset será utilizado con image_dataset_from_directory para cargar las imágenes al modelo.